In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train=np.expand_dims(x_train,axis=3)
x_test=np.expand_dims(x_test,axis=3)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

4431872/4422102 [==============================] - 0s 0us/step
(60000, 28, 28, 1)
(60000, 10)
(10000, 28, 28, 1)
(10000, 10)


In [3]:
class_names = [
   "T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt",
"Sneaker",
"Bag",
"Ankle boot"
]
AUTO = tf.data.AUTOTUNE
BATCH_SIZE = 64
EPOCHS = 10
IMG_SIZE=28

In [4]:
def preprocess_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.image.convert_image_dtype(image, tf.float32) / 255.0
    return image, label

In [5]:
train_ds_one = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .shuffle(1024)
    .map(preprocess_image, num_parallel_calls=AUTO)
)
train_ds_two = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .shuffle(1024)
    .map(preprocess_image, num_parallel_calls=AUTO)
)

train_ds_simple = tf.data.Dataset.from_tensor_slices((x_train, y_train))

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_ds_simple = (
    train_ds_simple.map(preprocess_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

# Combine two shuffled datasets from the same training data.
train_ds = tf.data.Dataset.zip((train_ds_one, train_ds_two))

test_ds = (
    test_ds.map(preprocess_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)


In [6]:
def sample_beta_distribution(size, concentration_0=0.2, concentration_1=0.2):
    gamma_1_sample = tf.random.gamma(shape=[size], alpha=concentration_1)
    gamma_2_sample = tf.random.gamma(shape=[size], alpha=concentration_0)
    return gamma_1_sample / (gamma_1_sample + gamma_2_sample)


@tf.function
def get_box(lambda_value):
    cut_rat = tf.math.sqrt(1.0 - lambda_value)

    cut_w = IMG_SIZE * cut_rat  # rw
    cut_w = tf.cast(cut_w, tf.int32)

    cut_h = IMG_SIZE * cut_rat  # rh
    cut_h = tf.cast(cut_h, tf.int32)

    cut_x = tf.random.uniform((1,), minval=0, maxval=IMG_SIZE, dtype=tf.int32)  # rx
    cut_y = tf.random.uniform((1,), minval=0, maxval=IMG_SIZE, dtype=tf.int32)  # ry

    boundaryx1 = tf.clip_by_value(cut_x[0] - cut_w // 2, 0, IMG_SIZE)
    boundaryy1 = tf.clip_by_value(cut_y[0] - cut_h // 2, 0, IMG_SIZE)
    bbx2 = tf.clip_by_value(cut_x[0] + cut_w // 2, 0, IMG_SIZE)
    bby2 = tf.clip_by_value(cut_y[0] + cut_h // 2, 0, IMG_SIZE)

    target_h = bby2 - boundaryy1
    if target_h == 0:
        target_h += 1

    target_w = bbx2 - boundaryx1
    if target_w == 0:
        target_w += 1

    return boundaryx1, boundaryy1, target_h, target_w


@tf.function
def cutmix(train_ds_one, train_ds_two):
    (image1, label1), (image2, label2) = train_ds_one, train_ds_two

    alpha = [0.25]
    beta = [0.25]

    # Get a sample from the Beta distribution
    lambda_value = sample_beta_distribution(1, alpha, beta)

    # Define Lambda
    lambda_value = lambda_value[0][0]

    # Get the bounding box offsets, heights and widths
    boundaryx1, boundaryy1, target_h, target_w = get_box(lambda_value)

    # Get a patch from the second image (`image2`)
    crop2 = tf.image.crop_to_bounding_box(
        image2, boundaryy1, boundaryx1, target_h, target_w
    )
    # Pad the `image2` patch (`crop2`) with the same offset
    image2 = tf.image.pad_to_bounding_box(
        crop2, boundaryy1, boundaryx1, IMG_SIZE, IMG_SIZE
    )
    # Get a patch from the first image (`image1`)
    crop1 = tf.image.crop_to_bounding_box(
        image1, boundaryy1, boundaryx1, target_h, target_w
    )
    # Pad the `image1` patch (`crop1`) with the same offset
    img1 = tf.image.pad_to_bounding_box(
        crop1, boundaryy1, boundaryx1, IMG_SIZE, IMG_SIZE
    )

    # Modify the first image by subtracting the patch from `image1`
    # (before applying the `image2` patch)
    image1 = image1 - img1
    # Add the modified `image1` and `image2`  together to get the CutMix image
    image = image1 + image2

    # Adjust Lambda in accordance to the pixel ration
    lambda_value = 1 - (target_w * target_h) / (IMG_SIZE * IMG_SIZE)
    lambda_value = tf.cast(lambda_value, tf.float32)

    # Combine the labels of both images
    label = lambda_value * label1 + (1 - lambda_value) * label2
    return image, label

In [8]:
import matplotlib.pyplot as plt

In [10]:
train_ds_cmu = (
    train_ds.shuffle(1024)
    .map(cutmix, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)


In [26]:
def get_training_model():
    model = tf.keras.Sequential(
        [
            layers.Conv2D(16, (5, 5), activation="relu", input_shape=(28, 28, 1)),
         
            layers.MaxPooling2D(pool_size=(4,4)),
            layers.BatchNormalization(),
            layers.Conv2D(32, (5, 5), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.BatchNormalization(),
            #layers.Dropout(0.2),
            layers.Flatten(),
            #layers.Dense(128, activation="relu"),
            layers.Dense(10, activation="softmax"),
        ]
    )
    return model


In [13]:
from keras import layers

In [27]:
initial_model = get_training_model()
initial_model.save_weights("initial_weights.h5")

In [32]:
model = get_training_model()
model.load_weights("initial_weights.h5")
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(train_ds_cmu,validation_data=test_ds, epochs=EPOCHS)
_, test_acc = model.evaluate(test_ds)
print("Test accuracy: {:.2f}%".format(test_acc * 100))

Epoch 1/10
938/938 [==============================] - 16s 17ms/step - loss: 1.1619 - accuracy: 0.6885 - val_loss: 0.4542 - val_accuracy: 0.8506
Epoch 2/10
938/938 [==============================] - 16s 17ms/step - loss: 1.0334 - accuracy: 0.7399 - val_loss: 0.4232 - val_accuracy: 0.8595
Epoch 3/10
938/938 [==============================] - 20s 21ms/step - loss: 1.0055 - accuracy: 0.7503 - val_loss: 0.4007 - val_accuracy: 0.8639
Epoch 4/10
938/938 [==============================] - 17s 18ms/step - loss: 0.9917 - accuracy: 0.7547 - val_loss: 0.3815 - val_accuracy: 0.8730
Epoch 5/10
938/938 [==============================] - 16s 17ms/step - loss: 0.9759 - accuracy: 0.7592 - val_loss: 0.3736 - val_accuracy: 0.8764
Epoch 6/10
938/938 [==============================] - 16s 17ms/step - loss: 0.9680 - accuracy: 0.7644 - val_loss: 0.3983 - val_accuracy: 0.8641
Epoch 7/10
938/938 [==============================] - 16s 17ms/step - loss: 0.9608 - accuracy: 0.7654 - val_loss: 0.3624 - val_accuracy:

In [33]:
model = get_training_model()
model.load_weights("initial_weights.h5")
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(train_ds_simple, validation_data=test_ds, epochs=10)

test_loss, test_accuracy = model.evaluate(test_ds)
print("Test accuracy: {:.2f}%".format(test_accuracy * 100))

Epoch 1/10
938/938 [==============================] - 6s 5ms/step - loss: 0.4527 - accuracy: 0.8427 - val_loss: 0.3856 - val_accuracy: 0.8641
Epoch 2/10
938/938 [==============================] - 5s 5ms/step - loss: 0.3116 - accuracy: 0.8891 - val_loss: 0.3717 - val_accuracy: 0.8712
Epoch 3/10
938/938 [==============================] - 5s 5ms/step - loss: 0.2793 - accuracy: 0.9005 - val_loss: 0.3886 - val_accuracy: 0.8672
Epoch 4/10
938/938 [==============================] - 5s 5ms/step - loss: 0.2598 - accuracy: 0.9061 - val_loss: 0.3756 - val_accuracy: 0.8721
Epoch 5/10
938/938 [==============================] - 5s 5ms/step - loss: 0.2445 - accuracy: 0.9119 - val_loss: 0.3640 - val_accuracy: 0.8788
Epoch 6/10
938/938 [==============================] - 5s 5ms/step - loss: 0.2331 - accuracy: 0.9151 - val_loss: 0.3767 - val_accuracy: 0.8745
Epoch 7/10
938/938 [==============================] - 5s 5ms/step - loss: 0.2220 - accuracy: 0.9193 - val_loss: 0.3775 - val_accuracy: 0.8744
Epoch 